In [2]:
import os 
import pandas as pd

In [3]:
data_dir = r"E:\Data\mimic-iv-3.0"
hosp = 'hosp'
icu = 'icu'

In [4]:
hosp_dir_path = os.path.join(data_dir, hosp)
icu_dir_path = os.path.join(data_dir,icu)

In [5]:
admissions = pd.read_csv(os.path.join(hosp_dir_path,'admissions.csv'))
icustays = pd.read_csv(os.path.join(icu_dir_path,'icustays.csv'))
labevents = pd.read_csv(os.path.join(hosp_dir_path, 'labevents.csv'))
chartevents = pd.read_csv(os.path.join(icu_dir_path, 'chartevents.csv'))

# 입원 기간 데이터 정리

In [ ]:
# datetime 형식으로 변환
admissions['admittime'] = pd.to_datetime(admissions['admittime'])
admissions['dischtime'] = pd.to_datetime(admissions['dischtime'])

# 입원 기간 동안의 날짜 생성
admissions['days_in_hospital'] = (admissions['dischtime'] - admissions['admittime']).dt.days


# 일별로 환자 데이터 생성

In [ ]:
# 각 환자마다 입원 기간 동안 날짜별로 데이터를 생성
all_dates = []
for index, row in admissions.iterrows():
    for i in range(row['days_in_hospital'] + 1):
        date = row['admittime'] + pd.Timedelta(days=i)
        all_dates.append([row['subject_id'], row['hadm_id'], date])

# DataFrame으로 변환
daily_data = pd.DataFrame(all_dates, columns=['subject_id', 'hadm_id', 'date'])


# 4. Vital Signs 및 Lab Results 결합

In [ ]:
# # 예시: 크레아티닌 수치 (lab_id가 50912인 경우)
creatinine_data = labevents[labevents['itemid'] == 50912]
creatinine_data['charttime'] = pd.to_datetime(creatinine_data['charttime'])
creatinine_data['date'] = creatinine_data['charttime'].dt.date

# creatinine_data와 daily_data를 병합
daily_data = pd.merge(daily_data, creatinine_data[['subject_id', 'date', 'valuenum']],
                      how='left', on=['subject_id', 'date'])

# 혈압 등의 vital signs 추가 (예시: itemid 220045는 혈압)
blood_pressure_data = chartevents[chartevents['itemid'] == 220045]
blood_pressure_data['charttime'] = pd.to_datetime(blood_pressure_data['charttime'])
blood_pressure_data['date'] = blood_pressure_data['charttime'].dt.date

# 베이스라인 크레아티닌 수치 계산 (입원 시 첫 크레아티닌 수치)
baseline = creatinine_data.groupby(['subject_id', 'hadm_id']).first().reset_index()
baseline = baseline.rename(columns={'valuenum': 'baseline_value', 'charttime': 'admission_time'})


# 데이터 병합
daily_data = pd.merge(daily_data, baseline[['subject_id', 'hadm_id', 'baseline_value']], on=['subject_id', 'hadm_id'])


# 5. AKI 라벨링 (KDIGO 기준)

In [1]:
# 절대 수치와 상대 수치 변화량 계산
daily_data['creatinine_change'] = daily_data['valuenum'] - daily_data['baseline_value']
daily_data['relative_change'] = (daily_data['valuenum'] / daily_data['baseline_value']) - 1


# KDIGO 기준 적용 (1.5배 이상 증가하거나 절대 수치가 0.3 이상 증가한 경우)
daily_data['AKI'] = (daily_data['creatinine_change'] >= 0.5).astype(int)
print(daily_data.head())


NameError: name 'daily_data' is not defined

In [ ]:
daily_data.to_csv('daily_data_icu.csv', index=False)